In [1]:
        !git clone https://github.com/AK18k/lora

Cloning into 'lora'...
remote: Enumerating objects: 1369, done.
remote: Counting objects: 100% (1369/1369), done.
remote: Compressing objects: 100% (975/975), done.
remote: Total 1369 (delta 365), reused 1358 (delta 359), pack-reused 0
Receiving objects: 100% (1369/1369), 23.86 MiB | 21.03 MiB/s, done.
Resolving deltas: 100% (365/365), done.


In [6]:
!pip install progress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9611 sha256=45621f354eb22cdf0cc34a3223fae441db596bf349b30b72992f4ed5526a554a
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
Successfully built progress


In [2]:
# !cd ./lora/examples/NLG
# !ls '*.sh'
!bash ./lora/examples/NLG/download_pretrained_checkpoints.sh

downloading pretrained model checkpoints...
--2023-06-03 09:17:48--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.103.158, 54.231.166.88, 52.217.161.144, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.103.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548118077 (523M) [application/octet-stream]
Saving to: ‘gpt2-pytorch_model.bin’

gpt2-pytorch_model. 100%[===================>] 522.73M  51.6MB/s    in 9.9s    

2023-06-03 09:17:58 (52.6 MB/s) - ‘gpt2-pytorch_model.bin’ saved [548118077/548118077]

--2023-06-03 09:17:58--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.106.14, 52.217.236.232, 52.217.83.238, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.106.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520013706 (1.4G) [appl

In [4]:
!bash ./lora/examples/NLG/create_datasets.sh

creating e2e datasets...
train...
Traceback (most recent call last):
  File "/content/lora/examples/NLG/src/gpt2_encode.py", line 31, in <module>
    from progress.bar import Bar as Bar
ModuleNotFoundError: No module named 'progress'
test...
Traceback (most recent call last):
  File "/content/lora/examples/NLG/src/gpt2_encode.py", line 31, in <module>
    from progress.bar import Bar as Bar
ModuleNotFoundError: No module named 'progress'
valid...
Traceback (most recent call last):
  File "/content/lora/examples/NLG/src/gpt2_encode.py", line 31, in <module>
    from progress.bar import Bar as Bar
ModuleNotFoundError: No module named 'progress'
^C
